<img src='./img/DataStore_EUMETSAT.png'/>

Copyright (c) 2024 EUMETSAT <br>
License: MIT

<hr>

# Accessing MTG LI Level 2 products

### Data used

| Product Description  | Data Store collection ID| Data Store |
|:--------------------:|:-----------------------:|:-----------------:|
| LI Accumulated Flashes - MTG - 0 degree | EO:EUM:DAT:0686 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0686) |
| LI Accumulated Flash Area - MTG - 0 degree | EO:EUM:DAT:0687 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0687) |
| LI Accumulated Flash Radiance - MTG - 0 degree | EO:EUM:DAT:0688 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0688) |
| LI Lightning Events - MTG - 0 degree | EO:EUM:DAT:0690 | [link](https://data.eumetsat.int/data/map/EO:EUM:DAT:0690)
| LI Lightning Flashes - MTG - 0 degree | EO:EUM:DAT:0691 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0691) |
| Lightning groups, full disc coverage - MTG | EO:EUM:DAT:0782 | [link](https://data.eumetsat.int/product/EO:EUM:DAT:0782) |

### Learning outcomes

At the end of this notebook you will know;
* How to refine your **searches** for MTG LI products in the EUMETSAT Data Store using the `eumdac` API client
* How to **download** products from your refined search results
* How to **convert** products into CSV format
* How to product's **convert epoch time** into datetime
* How MTG LI data is **structured and organised**

## Selecting a collection

As usual, we begin by importing our required modules.

In [1]:
import eumdac
import datetime
import shutil
import fnmatch
import requests
import time
import os
import glob
import zipfile
import pandas as pd
import netCDF4

Now, we have to authorize with our personal credentials to generate the token.

In [2]:
# Insert your personal key and secret into the single quotes
consumer_key = 'YOUR_CONSUMER_KEY'
consumer_secret = 'YOUR_CONSUMER_SECRET'
consumer_key = 'zbLPAxTQtOS5IZ9k0PrBLDlXGJwa'
consumer_secret = '6fSKOlY2fWG80gwBmHdT9mphDB4a'
credentials = (consumer_key, consumer_secret)

token = eumdac.AccessToken(credentials)

try:
    print(f"This token '{token}' expires {token.expiration}")
except requests.exceptions.HTTPError as error:
    print(f"Unexpected error: {error}")

This token 'cfbed384-aa58-33a6-ae4c-406340b88935' expires 2025-10-23 14:13:50.432065


<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
You can find your personal API credentials here: <a href="https://api.eumetsat.int/api-key/">https://api.eumetsat.int/api-key/</a>
</div>

The previous tutorial in this repository showing also how to list and find desired collections with EUMDAC. As we are focussing now on MTG LI collections, we are selecting the LI Lightning Flashes collection directly:

In [3]:
datastore = eumdac.DataStore(token)
selected_collection = datastore.get_collection('EO:EUM:DAT:0691')

*All publicly available MTG Lightning Imager collections, can also be in the [Data Store](https://data.eumetsat.int/extended?query=&filter=satellite__MTG&filter=instrument_type__Lightning%20Imager&filter=instrument__LI&filter=distribution__Download&filter=themes__Level%202%20Data).*

## Filtering and downloading products based on time interval

Previously we also covered how to filter collections for products of interest using both time and space. As MTG LI products are GEO products, there is no need for us to set a bounding box. However, we are going to perform a similar operation using time filtering so that we can set up a download. 

So, lets run our query to get the IDs for our products that are sensed within a full day (24 hour).

In [4]:
# Set date
date = datetime.date(2024, 10, 29)
#date = datetime.date(2025, 5, 15)
# Retrieve datasets that match our filter
products = selected_collection.search(
    dtstart=datetime.datetime.combine(date, datetime.time(0, 0)),
    dtend=datetime.datetime.combine(date, datetime.time(23, 59)),
    title=f"*_OPE_{date.strftime('%Y%m%d')}*_*")

print(f"{products.total_results} products found:")

for product in products:
    try:
        print(product)
    except eumdac.collection.CollectionError as error:
        print(f"Error related to the collection: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")

141 products found:
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241030000017_L2PF_OPE_20241029235003_20241030000003_N__C_0144_0000
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029235019_L2PF_OPE_20241029234003_20241029235003_N__C_0143_0000
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029234019_L2PF_OPE_20241029233003_20241029234003_N__C_0142_0000
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029233017_L2PF_OPE_20241029232003_20241029233003_N__C_0141_0000
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029232017_L2PF_OPE_20241029231003_20241029232003_N__C_0140_0000
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029231015_L2PF_OPE_20241029230003_20241029231003_N__C_0139_0000
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029230016_L2PF_OPE_20241029225003_20241029230003_N__C_0138_0000
W_XX-EUMETSAT-

In [5]:
selected_collection.search_options

{'bbox': {'title': 'Inventory which has a spatial extent overlapping this bounding box',
  'options': []},
 'geo': {'title': 'Inventory which has a spatial extent overlapping this Well Known Text geometry',
  'options': []},
 'title': {'title': 'Can be used to define a wildcard search on the product title (product identifier), use set notation as OR and space as AND operator between multiple search terms',
  'options': [None]},
 'sat': {'title': 'Mission / Satellite', 'options': ['MTI1']},
 'type': {'title': 'Product Type', 'options': ['MTILI2LFL']},
 'dtstart': {'title': 'Temporal Start', 'options': []},
 'dtend': {'title': 'Temporal End', 'options': []},
 'publication': {'title': 'publication date', 'options': []},
 'coverage': {'title': 'Coverage', 'options': ['FD']},
 'repeatCycleIdentifier': {'title': 'Repeat Cycle', 'options': []},
 'centerOfLongitude': {'title': 'Sub Sat Longitude', 'options': ['0.0']},
 'sort': {'title': 'SRU sort keys, see OpenSearch Extension for EO for detai

Now we have a list of all products acquired within a full day. In the following sections we will look at ways of downloading these.

The MTG Lighting Imager is continiously sensing. To make the data faster available to users, the products are published in chunks. The Data Store is publishing products with a chunk size of 10 minutes. Means, every 10 minutes a new MTG LI product get's published in EUMETSATs Data Store.

In the file name a 4-digit number (right-justified, zero-filled) indicating the current group repeat cycle in the day for this particular dataset. For LI, which does not have a natural repeat cycle, this is expected to correspond to the number of accumulation interval for the archived data (every 10 minutes in Data Store).

The counter starts at 0001 for the first group accumulation interval at or after midnight and resets for the next group accumulation interval at or after the following midnight.

This value can be used to associate all chunks of a repeat cycle dataset and aggregate them into a full day product.

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
The MTG products following the <i>WMO file naming convention</i>. You can find more information about this in our User Portal guide: <a href="https://user.eumetsat.int/resources/user-guides/mtg-li-level-2-data-guide#ID-Naming-convention">https://user.eumetsat.int/resources/user-guides/mtg-li-level-2-data-guide#ID-Naming-convention</a>
</div>

## Download products

We have found our relevant products. Now we would like to download all of them without having to select every single product. For this we can simply loop over the products and save the products as files in our directory.

In [6]:
# Define download directory
download_dir = 'downloads/'

In [9]:
for product in products:    
    try:
        with product.open() as fsrc, \
                open(download_dir + fsrc.name, mode='wb') as fdst:
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {product} finished.')
    except eumdac.product.ProductError as error:
        print(f"Error related to the product '{product}' while trying to download it: '{error.msg}'")
    except requests.exceptions.ConnectionError as error:
        print(f"Error related to the connection: '{error.msg}'")
    except requests.exceptions.RequestException as error:
        print(f"Unexpected error: {error}")
    
print('All downloads are finished.')

Download of product W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241030000017_L2PF_OPE_20241029235003_20241030000003_N__C_0144_0000 finished.
Download of product W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029235019_L2PF_OPE_20241029234003_20241029235003_N__C_0143_0000 finished.
Download of product W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029234019_L2PF_OPE_20241029233003_20241029234003_N__C_0142_0000 finished.
Download of product W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029233017_L2PF_OPE_20241029232003_20241029233003_N__C_0141_0000 finished.
Download of product W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029232017_L2PF_OPE_20241029231003_20241029232003_N__C_0140_0000 finished.
Download of product W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029231015_L2PF_OPE_20241029230003_20241029231003_N__C_0139_0000 fi

Perfect, now we downloaded all products of a full day, based on our search parameters.

<div class="alert alert-block alert-success">
<b>NOTE:</b><br />
Products can be downloaded by providing either their product ID, or a combination of their collection ID and several search parameters. We can download entire products, or specific file components (e.g. metadata only). This section was focussing on downloading the products from our search. To find more examples, see notebook <a href="./1_3_Downloading_products.ipynb">1.3 Downloading products</a>.
</div>

## Structure of products

A product consists of multiple files and are delivered as a [SIP](https://user.eumetsat.int/resources/user-guides/formats#ID-Submission-Information-Package-SIP). We can get the content of a product with the following command:

In [10]:
with zipfile.ZipFile(glob.glob("downloads/*.zip")[0], 'r') as zip_file:
    file_list = zip_file.namelist()
    for file in file_list:
        print(file)

W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--CHK-BODY--ARC-NC4E_C_EUMT_20241029200016_L2PF_OPE_20241029195003_20241029200003_N__C_0120_0001.nc
quicklooks/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--QCK-IMAGE--ARC-PNG_C_EUMT_20241029200016_L2PF_OPE_20241029195003_20241029200003_N__C_0120_0001.png
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--CHK-TRAIL--ARC-NC4E_C_EUMT_20241029200017_L2PF_OPE_20241029195003_20241029200003_N__C_0120_0002.nc
W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--QCK-IMAGE--ARC-PNG_C_EUMT_20241029200016_L2PF_OPE_20241029195003_20241029200003_N__C_0120_0001.jpg
manifest.xml
EOPMetadata.xml


As can be seen above, a product is made of the following files:
- **Body chunks (NetCDF)**: One or multiple body chunks (depending on product type) contain the measured data values in NetCDF structure.
- **Trailer chunk (NetCDF)**: One trailer chunk that contains at least the list of preceding body chunks that have been produced for the dataset. 
- **Quicklooks (PNG)**: Images related to data that enable fast viewing for the purpose of selection by archive users.
- **manifest.xml**: The manifest contains further information about the SIP especially the relationships among the different items constituting the package.
- **EOPMetadata.xml**: The EOPMetadata file contains metadata about the product based on the OGC Earth Observation Metadata Profile of Observations and Measurements (OGC document 10-157r4).

You can find more information about the [SIP files](https://user.eumetsat.int/resources/user-guides/formats#ID-Submission-Information-Package-SIP) and [MTG LI product/data structure](https://user.eumetsat.int/resources/user-guides/mtg-li-level-2-data-guide#ID-Level-2-data-characteristics) in the corresponding user guides.

That's it, now can we further process our data products.

## Extract the product SIP files

To further work with ther data, we have to uncompress the actual product content. This can simply be done with the following code:

In [11]:
Data_folder='downloads/'
#for file in glob.glob("downloads/*.zip"):
for file in glob.glob(Data_folder+"*.zip"):
    print(file)
    if os.path.exists(file):
        with zipfile.ZipFile(file, 'r') as zip_ref:
            # Extract the content of the .zip
            #zip_ref.extractall(path=f'{file[:-4]}/')
            zip_ref.extractall(Data_folder)
        # Remove the .zip file in favour of the extracted product
        os.remove(file)
        print(f"{file} has been unzipped and deleted.")
    else:
        print(f"{file} does not exist.")

downloads/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029200017_L2PF_OPE_20241029195003_20241029200003_N__C_0120_0000.zip
downloads/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029200017_L2PF_OPE_20241029195003_20241029200003_N__C_0120_0000.zip has been unzipped and deleted.
downloads/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029093020_L2PF_OPE_20241029092003_20241029093003_N__T_0057_0000.zip
downloads/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029093020_L2PF_OPE_20241029092003_20241029093003_N__T_0057_0000.zip has been unzipped and deleted.
downloads/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029151020_L2PF_OPE_20241029150003_20241029151003_N__C_0091_0000.zip
downloads/W_XX-EUMETSAT-Darmstadt,IMG+SAT,MTI1+LI-2-LFL--FD--x-x--ARC-x_C_EUMT_20241029151020_L2PF_OPE_20241029150003_20241029151003_N__C_0091_0000.zip has been unzipped and dele

## Converting values into a CSV file (optional)

As the MTG LI products are well structured in NetCDF files, we can convert them easily to text or a comma-seperated values (CSV) files. Depending on our use case, this can make it easier for other systems to work with the data.

First, we need to define which file we want to convert. To do so, we need to define the filename of the body chunk of our desired product and load the NetCDF file:

In [ ]:
file_to_convert = glob.glob("downloads/*BODY*.nc")[0]
li_ds = netCDF4.Dataset(file_to_convert, 'r')

Each product has a bunch of variables. To list all existing variables we can print them:

In [ ]:
for var in li_ds.variables:
    print(var)

Now, we can choose the variables we want to have in our CSV:

In [ ]:
latitude = li_ds['latitude'][:]
longitude = li_ds['longitude'][:]
flash_id = li_ds['flash_id'][:]
flash_time = li_ds['flash_time'][:]
flash_duration = li_ds['flash_duration'][:]
flash_footprint = li_ds['flash_footprint'][:]
radiance = li_ds['radiance'][:]
number_of_events = li_ds['number_of_events'][:]
number_of_groups = li_ds['number_of_groups'][:]
flash_filter_confidence = li_ds['flash_filter_confidence'][:]

The `flash_time` is measured as [epoch time](https://en.wikipedia.org/wiki/Epoch_%28computing%29) in products from Meteosat Third Generation satellites. To make it easier to work with them, we will recalculate the regular date time from it. We can do that, using the following function:

In [ ]:
def epoch_to_datetime(flash_time):
    flash_time_list = []
    
    for i in flash_time:
        i = int(i.flatten())
        mtg_reference_datetime=datetime.datetime(2000, 1, 1, 0, 0)
        mtg_datetime=mtg_reference_datetime + datetime.timedelta(seconds=i)
        flash_time_list.append(mtg_datetime)

    return flash_time_list

To bring them into a structure that fit's a CSV file, we are creating a dataframe where each observation is a row and every variable/value is in a seperate column.

In [ ]:
# Building dataframe
lfl = pd.DataFrame({
    'latitude': latitude.flatten(),
    'longitude': longitude.flatten(),
    'flash_id': flash_id.flatten(),
    'flash_time': epoch_to_datetime(flash_time),
    'flash_duration(ms)': flash_duration.flatten(),
    'flash_footprint(km2)': flash_footprint.flatten(),
    'radiance(scaled)': radiance.flatten(),
    'number_of_events': number_of_events.flatten(),
    'number_of_groups': number_of_groups.flatten(),
    'flash_filter_confidence(not_scaled)': flash_filter_confidence.flatten(),
})

Our dataframe is now generated:

In [ ]:
lfl

Finally, we can save them into a CSV file which is named as our product:

In [ ]:
lfl.to_csv(file_to_convert[:-3] + '.csv')

<hr>

<p style="text-align:left;">This project is licensed under the <a href="./LICENSE.TXT">MIT License</a> <span style="float:right;"><a href="https://gitlab.eumetsat.int/eumetlab/data-services/eumdac_data_store">View on GitLab</a> | <a href="https://classroom.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact</a></span></p>